In [2]:
import sys
sys.path.insert(1, '../src/')
import torch
from tqdm.notebook import tqdm
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets,transforms
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader
from torch.nn.utils import vector_to_parameters, parameters_to_vector
import matplotlib.pyplot as plt
import time
import os
import copy
import utils
from torchsummary import summary
import models
import random
import torch.nn.functional as F
import shutil
from torchsummary import summary
import torch
import torch.nn.functional as F
from PIL import Image
import os
import json
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
import torchvision
from torchvision import models
from torchvision import transforms
from captum.attr import LayerConductance
from captum.attr import IntegratedGradients
from captum.attr import GradientShap
from captum.attr import Occlusion
from captum.attr import NoiseTunnel
from captum.attr import visualization as viz
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from matplotlib import cm

In [3]:
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = True

In [ ]:
class args:
    model_file='iid_clean_model.pt'
    data='fmnist'
    bs = 256
    device = 'cuda:0'
    lr = 0.1
    moment = 0.9
    wd = 0
    epochs=50
    nesterov = True
    base_class = 5
    target_class = 7
    poison_frac = 0.05
    pattern_type='+'

In [ ]:
train_dataset, val_dataset = utils.get_datasets(args.data)
idxs = (val_dataset.targets == args.base_class).nonzero().flatten().tolist()
poisoned_val_set = utils.DatasetSplit(copy.deepcopy(val_dataset), idxs)
utils.poison_dataset(poisoned_val_set.dataset, args, poison_all=True)

In [ ]:
val_loader =  DataLoader(val_dataset, batch_size=len(val_dataset), shuffle=False, num_workers=2, pin_memory=True)
poisoned_val_loader = DataLoader(poisoned_val_set, batch_size=args.bs, shuffle=False, num_workers=0, pin_memory=True) 

In [ ]:
model_clean = torch.load(args.model_file).eval()
model_adv = torch.load('iid_nodefense_model.pt').eval()
criterion = nn.CrossEntropyLoss().to(args.device)

In [ ]:
model_clean

In [ ]:
_, (imgs, labels) = next(enumerate(poisoned_val_loader))
idx = (labels==args.target_class).nonzero()[25].item()

In [ ]:
img = imgs[idx]
plt.imshow(img.squeeze(0).cpu().numpy())

In [ ]:
img = img.view(1, 1, 28, 28).to(args.device)

In [ ]:
output = model_adv(img)

In [ ]:
output = F.softmax(output, dim=1)
prediction_score, pred_label_idx = torch.topk(output, 1)

pred_label_idx.squeeze_()
predicted_label = pred_label_idx.item()
print('Predicted:', predicted_label, '(', prediction_score.squeeze().item(), ')')

In [ ]:
clean_filter_importance = []
layer_cond = LayerConductance(model_clean, model_clean.conv2)
attribution_clean = layer_cond.attribute(img, target=5)[0]
for flt in attribution_clean:
    flt_importance = flt.flatten().sum().item()
    clean_filter_importance.append(flt_importance)


In [ ]:
adv_filter_importance = []
layer_cond = LayerConductance(model_adv, model_adv.conv2)
attribution_adv = layer_cond.attribute(img, target=7)[0]
for flt in attribution_adv:
    flt_importance = flt.flatten().sum().item()
    adv_filter_importance.append(flt_importance)

In [ ]:
np.argsort(clean_filter_importance)

In [ ]:
np.argsort(adv_filter_importance)

In [ ]:
model_adv.conv2.weight[0]

In [ ]:
layer_cond = LayerConductance(model_adv, model_adv.conv2)
attribution_adv = layer_cond.attribute(img, target=7)

In [ ]:
attribution_adv

In [21]:
t = torch.Tensor([1, 3, -1, 2]).cuda()
t2 = torch.Tensor([1, -4, 0, -2]).cuda()
t.sign() ^ t2.sign()

RuntimeError: "bitwise_xor_cuda" not implemented for 'Float'

In [32]:
(t.sign() == t2.sign()).sum().item()/5

0.2

torch.Size([4])

RuntimeError: "bitwise_xor_cuda" not implemented for 'Float'